In [124]:
import geopandas as gpd
import pandas as pd

In [125]:
# schools contains the point coordinates of all schools in Berlin
schools = gpd.read_file('raw/schulen.geojson')

# we want to find the respective plr for each school, so we need the polygons of the plr
plr = gpd.read_file('../plr/plr_only.geojson')

# plr without geometry
plr_no_geo = plr.drop(columns='geometry').sort_values(by='PLR_ID')

# check for same crs (coordinate reference system)
schools.crs == plr.crs

True

In [126]:
# Spatial join of schools and plr
schools_with_plr = gpd.sjoin(schools, plr, how='inner', predicate='within')

print("schools shape:", schools.shape, "\nschools_with_plr shape:", schools_with_plr.shape)

schools shape: (902, 15) 
schools_with_plr shape: (902, 18)


In [127]:
schools_with_plr.columns

Index(['name', 'bsn', 'schulart', 'traeger', 'schultyp', 'bezirk', 'ortsteil',
       'plz', 'strasse', 'hausnr', 'telefon', 'fax', 'email', 'internet',
       'geometry', 'index_right', 'PLR_ID', 'PLR_NAME'],
      dtype='object')

In [128]:
# only keep relevant features, get rid of geometry -> regular pandas dataframe
feature_list = ['name', 'schulart', 'traeger', 'schultyp', 'PLR_ID']
df = schools_with_plr[feature_list]

In [129]:
df

,name,schulart,traeger,schultyp,PLR_ID
0,"OSZ Banken, Immobilien und Versicherungen",Oberstufenzentrum,öffentlich,Berufsschule,01200627
1,Staatliche Technikerschule Berlin,Fachschule,öffentlich,Berufsschule,01200522
36,Miriam-Makeba-Grundschule,Grundschule,öffentlich,Grundschule,01200522
2,"OSZ Kommunikations-, Informations- und Medient...",Oberstufenzentrum,öffentlich,Berufsschule,01300730
21,Wilhelm-Hauff-Grundschule,Grundschule,öffentlich,Grundschule,01300730
...,...,...,...,...,...
865,Jean-Krämer-Schule (Integrierte Sekundarschule...,Integrierte Sekundarschule,öffentlich,Integrierte Sekundarschule,12500926
890,Schule am Park,"Förderschwerp. ""Geistige Entwicklung""",öffentlich,Schule mit sonderpädagogischem Förderschwerpunkt,12500926
874,Private Goethe-Schulen,Kombinierte allgemein bildende Schule,privat,Privatschule,12601031
875,Demokratische Schule X (Gemeinschaftsschule),Gemeinschaftsschule,privat,Privatschule,12400618


In [130]:
# We want to get a summary for each plr, so we group by plr_id...
gb = df.groupby('PLR_ID')

school_count = gb.size().reset_index(name='school_count')
school_names = gb['name'].apply(list).reset_index(name='school_names')
schulart = gb['schulart'].value_counts().unstack().reset_index().fillna(0)
schultyp = gb['schultyp'].value_counts().unstack().reset_index().fillna(0)
gb_traeger = gb['traeger'].value_counts().unstack().reset_index().fillna(0)

In [131]:
schulart.columns

Index(['PLR_ID', 'Berufsfachschule', 'Berufsschule',
       'Berufsschule mit sonderpäd. Aufgaben', 'Fachschule',
       'Freie Waldorfschule', 'Förderschwerp. "Geistige Entwicklung"',
       'Förderschwerp. "Lernen"',
       'Förderschwerp. "Lernen"u."Geistige Entwickl."',
       'Förderschwerp. einschl. beruflichem Teil', 'Gemeinschaftsschule',
       'Grundschule', 'Gymnasium', 'Integrierte Sekundarschule',
       'Kombinierte allg./berufl. Schule',
       'Kombinierte allgemein bildende Schule',
       'Kombinierte berufliche Schule', 'Oberstufenzentrum',
       'Übrige Förderschwerpunkte'],
      dtype='object', name='schulart')

In [132]:
schultyp.columns

Index(['PLR_ID', 'Andere Schule', 'Berufsschule', 'Grundschule', 'Gymnasium',
       'Integrierte Sekundarschule', 'Privatschule',
       'Schule mit sonderpädagogischem Förderschwerpunkt'],
      dtype='object', name='schultyp')

In [133]:
# For the verbose frame, we need to prefix schulart and schultyp, as they have
# some equal names which would be confusing...
# First columns is PLR_ID, we want to keep that the same as merge key later...

cols_schulart_prefixed = ['PLR_ID'] + ['Art: ' + col for col in schulart.columns[1:]]
schulart_prefixed = schulart.copy()
schulart_prefixed.columns = cols_schulart_prefixed

cols_schultyp_prefixed = ['PLR_ID'] + ['Typ: ' + col for col in schultyp.columns[1:]]
schultyp_prefixed = schultyp.copy()
schultyp_prefixed.columns = cols_schultyp_prefixed

In [134]:
# Merge to simple summary

df_simple = plr_no_geo \
    .merge(school_count, on='PLR_ID', how='outer') \
    .merge(school_names, on='PLR_ID', how='outer') \
    .merge(gb_traeger, on='PLR_ID', how='outer') \
    .merge(schultyp, on='PLR_ID', how='outer')


df_verbose = plr_no_geo \
    .merge(school_count, on='PLR_ID', how='outer') \
    .merge(school_names, on='PLR_ID', how='outer') \
    .merge(gb_traeger, on='PLR_ID', how='outer') \
    .merge(schultyp_prefixed, on='PLR_ID', how='outer') \
    .merge(schulart_prefixed, on='PLR_ID', how='outer')

In [137]:
df_simple.head()

,PLR_ID,PLR_NAME,school_count,school_names,privat,öffentlich,Andere Schule,Berufsschule,Grundschule,Gymnasium,Integrierte Sekundarschule,Privatschule,Schule mit sonderpädagogischem Förderschwerpunkt
0,01100101,Stülerstraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01100102,Großer Tiergarten,1.0,[Canisius-Kolleg ],1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,01100103,Lützowstraße,4.0,"[Allegro-Grundschule, Internationale Lomonosso...",2.0,2.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0
3,01100104,Körnerstraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01100205,Wilhelmstraße,1.0,[Grundschule am Brandenburger Tor],0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [139]:
df_verbose.head()

,PLR_ID,PLR_NAME,school_count,school_names,privat,öffentlich,Typ: Andere Schule,Typ: Berufsschule,Typ: Grundschule,Typ: Gymnasium,...,Art: Förderschwerp. einschl. beruflichem Teil,Art: Gemeinschaftsschule,Art: Grundschule,Art: Gymnasium,Art: Integrierte Sekundarschule,Art: Kombinierte allg./berufl. Schule,Art: Kombinierte allgemein bildende Schule,Art: Kombinierte berufliche Schule,Art: Oberstufenzentrum,Art: Übrige Förderschwerpunkte
0,01100101,Stülerstraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01100102,Großer Tiergarten,1.0,[Canisius-Kolleg ],1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,01100103,Lützowstraße,4.0,"[Allegro-Grundschule, Internationale Lomonosso...",2.0,2.0,0.0,0.0,1.0,1.0,...,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01100104,Körnerstraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01100205,Wilhelmstraße,1.0,[Grundschule am Brandenburger Tor],0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
# Save to csv
df_simple.to_csv('schools_per_plr_simple.csv', index=False)
df_verbose.to_csv('schools_per_plr_verbose.csv', index=False)